In [58]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import integrate
import geopy.distance
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import requests
import time

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [18]:
cred = credentials.Certificate('./../Backend/serviceAccount.json')
app_firebase = firebase_admin.initialize_app(cred)
db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [19]:
import sys
 
sys.path.append('./../Backend/')

In [20]:

 
# importing the add and odd_even
# function
from key_value.token import apiKey, apiUrl, auth_header


 


In [21]:
auth_header

{'x-apikey': 'IaF6zOUaj8VG4ambmUwOodEFaxBnrR3A'}

In [61]:
path = './ONLY_TEST/'
csv_files = glob.glob(os.path.join(path, "*.csv"))
for f in csv_files:
    # print(f)
    if os.name == 'nt':
        filename = f.split('\\')[1]
    else:
        filename = f.split('/')[2]
        
    flight_id = filename.split('-')[0]
    print(flight_id)
    doc_ref = db.collection('aircraft_engine').document(f'{flight_id}')
    docs = doc_ref.get()

    if docs.exists:
        print("Found")
        print(docs.to_dict())
    else:
        print("Not found")
        #----- get aircraft_type from flight_ID -----#
        response = requests.get(apiUrl + f"flights/{flight_id}",headers=auth_header)

        if response.status_code == 200:
            # print("got aircraft_type")
            aircraft_type = response.json()['flights'][0]['aircraft_type']
            
            #----- get engine data from aircraft_type -----#
            response = requests.get(apiUrl + f"aircraft/types/{aircraft_type}",headers=auth_header)
            
            if response.status_code == 200:
                # print("got engine data")
                engine_data = response.json()
                engine_data.pop('manufacturer', None)
                engine_data.pop('description', None)
                engine_data['flight_id'] = flight_id
                
                #----- add to firestore -----#
                doc_ref = db.collection(f'aircraft_engine').document(f'{flight_id}')
                doc_ref.set(engine_data)
                
            else:
                print("Error executing request")
            
        else:
            print("Error executing request")
        
        time.sleep(10)




AIQ3029
Found
{'type': 'A320', 'flight_id': 'AIQ3029', 'engine_type': 'jet', 'engine_count': 2}
AIQ3102
Found
{'type': 'A320', 'flight_id': 'AIQ3102', 'engine_type': 'jet', 'engine_count': 2}
AIQ311
Found
{'type': 'A320', 'flight_id': 'AIQ311', 'engine_type': 'jet', 'engine_count': 2}
AIQ3188
Found
{'type': 'A320', 'flight_id': 'AIQ3188', 'engine_type': 'jet', 'engine_count': 2}
AIQ3252
Found
{'type': 'A320', 'flight_id': 'AIQ3252', 'engine_type': 'jet', 'engine_count': 2}
AIQ3362
Found
{'type': 'A320', 'flight_id': 'AIQ3362', 'engine_type': 'jet', 'engine_count': 2}
AIQ3437
Not found
AIQ512
Found
{'type': 'A320', 'flight_id': 'AIQ512', 'engine_type': 'jet', 'engine_count': 2}
NOK100
Not found
NOK122
Not found
NOK340
Not found
NOK570
Not found
TLM702
Found
{'type': '737-800', 'flight_id': 'TLM702', 'engine_type': 'jet', 'engine_count': 2}


# read NPD_ID

In [72]:
path = './ONLY_TEST/'
csv_files = glob.glob(os.path.join(path, "*.csv"))
for f in csv_files:
    # print(f)
    if os.name == 'nt':
        filename = f.split('\\')[1]
    else:
        filename = f.split('/')[2]
        
    flight_id = filename.split('-')[0]
    print(flight_id)
    
    #----- find ACFT_ID -----#
    acft_ref = db.collection(f'aircraft_engine').document(f'{flight_id}')
    acft_docs = acft_ref.get()

    if acft_docs.exists:
        # print("Found")
        acft_docs = acft_docs.to_dict()
        ACFT_ID = acft_docs['type']
        # print(ACFT_ID)
    else:
        # print("Not found")
        continue
    
    #----- find NPD_ID -----#
    npd_ref = db.collection(f'NPD_ID').document(f'{ACFT_ID}')
    npd_docs = npd_ref.get()
    
    if npd_docs.exists:
        # print("Found")
        npd_docs = npd_docs.to_dict()
        NPD_ID = npd_docs['NPD_ID']
        print(NPD_ID)
    else:
        print("Not found")
        continue
    
    


AIQ3029
CFM565
AIQ3102
CFM565
AIQ311
CFM565
AIQ3188
CFM565
AIQ3252
CFM565
AIQ3362
CFM565
AIQ3437
CFM565
AIQ512
CFM565
NOK100
CF567B
NOK122
CF567B
NOK340
PW120
NOK570
CF567B
TLM702
CF567B
